<a href="https://colab.research.google.com/github/jooalee64/mastertip/blob/main/mastertip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torchvision.models as models

class MastTipDetectionModel(nn.Module):
    def __init__(self, num_classes=1):
        super(MastTipDetectionModel, self).__init__()
        # Using a pre-trained ResNet as a backbone (example)
        self.backbone = models.resnet50(pretrained=True)
        # Modify the classifier for bounding box regression, mast tip prediction, and uncertainty
        num_ftrs = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity() # Remove the original classification layer

        # Add custom layers for the outputs
        # Bounding box: 4 coordinates (x1, y1, x2, y2)
        self.bbox_predictor = nn.Linear(num_ftrs, 4)
        # Mast tip: 2 coordinates (x, y)
        self.mast_tip_predictor = nn.Linear(num_ftrs, 2)
        # Uncertainty: 1 value (scalar)
        self.uncertainty_predictor = nn.Linear(num_ftrs, 1)

    def forward(self, x):
        # Forward pass through the backbone
        features = self.backbone(x)

        # Predict outputs
        bbox = self.bbox_predictor(features)
        mast_tip = self.mast_tip_predictor(features)
        uncertainty = self.uncertainty_predictor(features)

        return bbox, mast_tip, uncertainty

# Example of model instantiation (without data)
# model = MastTipDetectionModel()
# print(model)

This code defines a basic PyTorch model `MastTipDetectionModel` that uses a pre-trained ResNet50 as a backbone. It adds custom linear layers to predict:
- `bbox`: A bounding box with 4 coordinates.
- `mast_tip`: The mast tip coordinates with 2 values.
- `uncertainty`: A single value representing uncertainty.

The `forward` method shows how an input image `x` would be processed to produce these outputs.

Remember, this is a simplified example. The actual implementation based on the paper might require a more complex architecture, custom loss functions, and specific handling of the uncertainty prediction.